# 02 형식언어(Formal Languages)

한남대학교 2019 봄(1학기) 오토마타 이론 (교육대학원 전자계산교육전공 과목) 강의 활용 자료입니다.

----
[노엄 촘스키](https://ko.wikipedia.org/wiki/%EB%85%B8%EC%97%84_%EC%B4%98%EC%8A%A4%ED%82%A4)([Noam Chomsky](https://en.wikipedia.org/wiki/Noam_Chomsky))가
제안한 변형생성문법(transformational-generative grammar), 줄여서 생성문법(generative grammar)이라고도 하고 초기에는 변형문법(transformational grammar)라고 불렸다고 한다.
노엄 촘스키의 업적은 언어학에서 혁명적인 사건으로 평가받고 있으며 언어학 뿐 아니라 형식언어를 연구하는 컴퓨터과학 이론과 활용에 기초적 발판을 마련해 준 위대한 업적이다.
책에서는 앞으로 이 과목에서 형식언어를 다루는 데 있어 이와 관련된 내용을 간략하게 소개하고 있다.

관련 에피소드로 "유명한 컴퓨터 과학자인 도널드 커누스는 신혼여행동안 이 책을 읽었고, 이 책에 많은 영향을 받았다고 그의 2003년도 책인 Selected Papers on Computer Languages의 서문을 통해 밝혔다"고 한다.
출처: https://ko.wikipedia.org/wiki/%EB%B3%80%ED%98%95%EC%83%9D%EC%84%B1%EB%AC%B8%EB%B2%95%EC%9D%98_%EC%9D%B4%EB%A1%A0

## 2.1 규칙(rule)에 의한 언어 생성

봄-문법:
 * $\langle 문장\rangle \to \langle 주어\rangle~\langle 술어\rangle$
 * $\langle 주어\rangle \to 새가$
 * $\langle 주어\rangle \to 꽃이$
 * $\langle 주어\rangle \to 싹이$
 * $\langle 술어\rangle \to 노래한다$
 * $\langle 술어\rangle \to 핀다$
 * $\langle 술어\rangle \to 난다$

위 문법을 줄여서 간단히 쓰기 위해 $\mid$ 기호로 다음과 같이 나타낸다.
 * $\langle 문장\rangle \to \langle 주어\rangle~\langle 술어\rangle$
 * $\langle 주어\rangle \to 새가 \mid 꽃이 \mid 싹이$
 * $\langle 술어\rangle \to 노래한다 \mid 핀다 \mid 난다$

위 규칙으로 생성할 수 있는 문장의 한 예로 다음과 같은 것이 있다.

$\langle 문장\rangle \Rightarrow \langle 주어\rangle~\langle 술어\rangle \Rightarrow 새가~\langle 술어\rangle \Rightarrow 새가~노래한다$

이론을 전개할 때는 간략하게 한 글자 알파벳 기호로 표현하는 경우가 많다.
봄-문법을 $G$로 나타내고 다음과 같이 각 요소를 대응시키자.
 * $S : \langle 문장\rangle$
 * $A : \langle 주어\rangle$
 * $B : \langle 동사\rangle$
 * $a : 새가$
 * $b : 꽃이$
 * $c : 싹이$
 * $d : 노래한다$
 * $e : 핀다$
 * $f : 난다$

이제 $G$를 다시 알파벳 기호로 다시 쓰자면 다음과 같다.
 * $G \to AB$
 * $A \to a \mid b \mid c$
 * $B \to d \mid e \mid f$

위 규칙으로 생성할 수 있는 문자열의 예로 다음과 같은 것이 있다.

$S \Rightarrow AB \Rightarrow aB \Rightarrow ad$

여기서 생성된 문자열 $ab$는 $``새가~노래한다"$는 문장에 해당한다.

In [1]:
data Nterm = S | A | B
  deriving (Eq,Ord,Show)

data Gelem = N Nterm | T String
  deriving (Eq,Ord,Show)

data Rule = Nterm :-> [Gelem]
  deriving (Eq,Ord,Show)

type Grammar = [Rule]

_G :: Grammar
_G = [ S :-> [N A, N B]
     , A :-> [T "a"], A :-> [T "b"], A :-> [T "c"]
     , B :-> [T "d"], B :-> [T "e"], B :-> [T "f"]
     ]

(==>) :: [Gelem] -> [Gelem] -> Grammar -> Bool
es ==> es' = \rs -> es' `elem` [applyRuleAt r i es | r <- rs, i <- [0..length es-1]]

applyRuleAt :: Rule -> Int -> [Gelem] -> [Gelem]
applyRuleAt r i es = concat [if i==j then (applyRule r e) else [e] | (j,e) <- zip [0..] es]

applyRule :: Rule -> Gelem -> [Gelem]
applyRule (_A :-> es) (N _A') | _A == _A' = es
applyRule (_A :-> es) e                   = [e]

아래는 하스켈로 $S \Rightarrow AB \Rightarrow aB \Rightarrow ad$ 과정을 단계별로 검산한 결과이다.

In [2]:
[N S]       ==> [N A,  N B]  $ _G
[N A,  N B] ==> [T"a", N B]  $ _G
[T"a", N B] ==> [T"a", T"d"] $ _G

True

True

True

In [4]:
t2w (T"a") = "새가"
t2w (T"b") = "꽃이"
t2w (T"c") = "싹이"
t2w (T"d") = "노래한다"
t2w (T"e") = "핀다"
t2w (T"f") = "난다"

In [5]:
putStrLn . unwords $ map t2w [T"a", T"d"]

새가 노래한다

## 2.2 정의(Definition): 형식언어(formal language)와 문법(grammar)

촘스키 계층구조(Chomsky hierarchy): 언어를 생성하는 문법의 자유도에 따른 언어의 분류 체계

문법은 다음과 같이 네 요소로 정의된다:
$G = \langle V_T, V_N, P, S \rangle$.

* $V_T$: 단말 기호(terminal symbol)의 집합. 단말 알파벳(terminal alphabet). 관례적으로 소문자로 나타낸다. 언어학에서는 형태소(morphemes)라고도 함.
* $V_N$: 비단말 기호(nonterminal symbol)의 집합. 비단말. 관례적으로 대문자로 나타낸다. 언어학에서는 통사범주(syntactic category)라고도 함.
* $V = V_T \cup V_N$. 당연히 $V_T \cup V_N = \varnothing$임을 가정한다.
* $S \in V_T$: 시작 기호(start symbol).
* $P \subset \{ \alpha \to \beta \mid \alpha\in V^{*} V_N V^{*},~\beta\in V^{*} \}$. 문법규칙의 유한한 집합이어야 한다 (즉, $|P| \in \mathbb{N}$).

문법 $G$로 정의되는 언어 $L(G) = \{x | S \Rightarrow^{*} x\in {V_T}^{*} \}$.

참고로 $V_T$ 대신 $\Sigma$, 그리고 $V_N$대신 $N$으로 표기하기도 한다.
($G = \langle \Sigma, V_N, P, S \rangle ~\text{혹은}~ G = \langle \Sigma, N, P, S\rangle$.)

#### type-0: 재귀열거(가능)언어 (recursively enumerable language)
구문구조문법(phrase structured grammar)으로 정의(또는 생성) 가능한 언어.

문법규칙에 제약조건 없음.

#### type-1: 문맥의존언어 (context-sensitive language, CSL)
문맥의존문법(context-sensitive grammar, CSG)으로 정의 가능한 언어.

$\alpha A \beta \to \alpha \gamma \beta$ 형태의 문법규칙만 가능.

#### type-2: 문맥자유언어 (context-free langage, CFL)
문맥자유문법(context-senstivie grammar, CFG)으로 정의 가능한 언어.

문법규칙($\alpha\to\beta$)에서 $|\alpha|=1$, 즉 $\alpha\in V_N$이어야 한다.
즉 $A \to \beta$ 형태의 문법규칙만 가능.

#### type-3: 정규언어 (regular language)
정규문법(regular grammar)으로 정의 가능한 언어.

$A\to \varepsilon$, $A \to a$, $A \to aB$ 형태의 문법규칙만 가능.